In [ ]:
from fastai.text import * 

# Wikitext 103

In [ ]:
path = Config().data_path()/'wikitext-103'

In [ ]:
def istitle(line):
    return len(re.findall(r'^ = [^=]* = $', line)) != 0

In [ ]:
def read_file(filename):
    articles = []
    with open(filename, encoding='utf8') as f:
        lines = f.readlines()
    current_article = ''
    for i,line in enumerate(lines):
        current_article += line
        if i < len(lines)-2 and lines[i+1] == ' \n' and istitle(lines[i+2]):
            current_article = current_article.replace('<unk>', UNK)
            articles.append(current_article)
            current_article = ''
    current_article = current_article.replace('<unk>', UNK)
    articles.append(current_article)
    return np.array(articles)

In [ ]:
train = read_file(path/'train.txt')
valid = read_file(path/'valid.txt')
test =  read_file(path/'test.txt')

In [ ]:
len(train), len(valid), len(test)

In [ ]:
all_texts = np.concatenate([valid, train, test])
df = pd.DataFrame({'texts':all_texts})
df.head()

In [ ]:
del train
del valid
del text

In [ ]:
data = (TextList.from_df(df, path, cols='texts')
                .split_by_idx(range(0,60))
                .label_for_lm()
                .databunch(bs=100, bptt=70))

In [ ]:
data.show_batch()

In [ ]:
data.save()

In [ ]:
data = load_data(path)
data.show_batch()

In [ ]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.1, pretrained=False, clip=0.1, 
                               metrics=[accuracy, Perplexity()])

In [ ]:
learn.fit_one_cycle(10, 5e-3, moms=(0.8,0.7), div_factor=10, wd=1e-3)

In [ ]:
learn.save('lstm', with_opt=False)

In [ ]:
learn.data.vocab.save(path/'vocab.pkl')